# 通过CNN分析价格曲线，预测涨跌  
 
20 x 20=400个工作日  20个月 80个礼拜  => if 未来20个工作日的平均股价 > 当前股价


In [2]:
import baostock as bs
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import matplotlib. pyplot as plt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape,Dropout,Activation
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [1]:
import datetime
import baostock as bs

In [3]:
stock_code = "sz.000858"
stock_info_path = "stock_info/" + stock_code + ".csv"
stock = pd.read_csv(stock_info_path, parse_dates=['date'])
stock = stock[stock['date'] < datetime.datetime.now()- datetime.timedelta(days=20)]
stock

,date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn
0,2010-01-04,sz.000858,31.80,32.00,31.36,31.45,31.66,26409039,8.363519e+08,3,0.695795
1,2010-01-05,sz.000858,31.48,31.95,31.48,31.78,31.45,29098216,9.237874e+08,3,0.766646
2,2010-01-06,sz.000858,31.78,31.98,30.90,30.94,31.78,42822040,1.340543e+09,3,1.128226
3,2010-01-07,sz.000858,30.78,30.78,29.49,30.00,30.94,61705156,1.850036e+09,3,1.625736
4,2010-01-08,sz.000858,29.85,29.96,29.09,29.94,30.00,36477184,1.075206e+09,3,0.961059
...,...,...,...,...,...,...,...,...,...,...,...
2728,2021-03-26,sz.000858,258.00,267.18,255.82,265.95,255.98,23020269,6.054732e+09,3,0.606500
2729,2021-03-29,sz.000858,267.00,279.95,265.99,272.09,265.95,28199481,7.712654e+09,3,0.742900
2730,2021-03-30,sz.000858,270.50,278.52,270.50,273.46,272.09,16870779,4.624076e+09,3,0.444500
2731,2021-03-31,sz.000858,270.70,270.70,265.01,267.98,273.46,13411104,3.583946e+09,3,0.353300


In [16]:
list(stock.loc[0])

[Timestamp('2010-01-04 00:00:00'),
 'sz.000858',
 31.8,
 32.0,
 31.36,
 31.45,
 31.66,
 26409039,
 836351949.35,
 3,
 0.695795]

In [55]:
for i in range(20, 0, -1):
    print(i)

20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [11]:
bs.login()

stock_code = "sz.000858"
stock_info_path = "stock_info/" + stock_code + ".csv"

to_date = datetime.datetime.now().strftime("%Y-%m-%d")
# to_date = '2021-04-18'          # 今日日期
re_download = True              # 重新下载
re_train = True                # 是否训练
predict_period = 20             # 预测天数
history_period = 400            # 分析天数
epoch = 200
start_date = '2010-01-01'
rs = bs.query_history_k_data(stock_code, "date, close, volume, amount, turn", start_date=start_date, end_date=to_date, frequency="d", adjustflag="3")
data_list = []
while (rs.error_code == '0') & rs.next():  # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
result.to_csv(stock_info_path, index=False)

login success!


In [26]:
stock_info_path = "stock_info/sz.000858.csv"
stock = pd.read_csv(stock_info_path, parse_dates=['date'])

In [4]:
predict_period = 5
history_period = 400

In [34]:
stock['close_nomalized'] = (stock['close']-stock['close'].min())/(stock['close'].max()-stock['close'].min())
stock['volume_nomalized'] = (stock['volume']-stock['volume'].min())/(stock['volume'].max()-stock['volume'].min())
stock['future_price'] = stock['close'].rolling(predict_period).mean().shift(-predict_period)
stock['avg_price'] = stock['close'].rolling(predict_period).mean()

In [35]:
stock

,date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,close_nomalized,future_price,avg_price,volume_nomalized
0,2010-01-04,sz.000858,31.80,32.00,31.36,31.45,31.66,26409039,8.363519e+08,3,0.695795,0.051465,30.464,NaN,0.145460
1,2010-01-05,sz.000858,31.48,31.95,31.48,31.78,31.45,29098216,9.237874e+08,3,0.766646,0.052466,30.210,NaN,0.160272
2,2010-01-06,sz.000858,31.78,31.98,30.90,30.94,31.78,42822040,1.340543e+09,3,1.128226,0.049920,30.076,NaN,0.235862
3,2010-01-07,sz.000858,30.78,30.78,29.49,30.00,30.94,61705156,1.850036e+09,3,1.625736,0.047071,30.364,NaN,0.339869
4,2010-01-08,sz.000858,29.85,29.96,29.09,29.94,30.00,36477184,1.075206e+09,3,0.961059,0.046889,30.688,30.822,0.200915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,2021-04-12,sz.000858,265.20,267.36,260.00,260.99,265.30,13323255,3.495255e+09,3,0.351000,0.747189,NaN,270.996,0.073384
2739,2021-04-13,sz.000858,261.26,268.22,261.26,262.90,260.99,13875729,3.670700e+09,3,0.365600,0.752978,NaN,266.624,0.076427
2740,2021-04-14,sz.000858,262.91,267.61,261.25,263.78,262.90,14817752,3.916715e+09,3,0.390400,0.755645,NaN,265.210,0.081615
2741,2021-04-15,sz.000858,261.31,262.99,249.32,255.80,263.78,28595938,7.267459e+09,3,0.753400,0.731458,NaN,261.754,0.157505


In [39]:
stock[stock.isnull().T.any()][-10:]

,date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,close_nomalized,future_price,avg_price,volume_nomalized
1407,2015-10-26,sz.000858,25.73,25.73,25.73,25.73,25.73,0,0.000000e+00,3,NaN,0.034128,25.424,25.730,0.000000
1408,2015-10-27,sz.000858,25.73,25.73,25.73,25.73,25.73,0,0.000000e+00,3,NaN,0.034128,25.092,25.730,0.000000
1409,2015-10-28,sz.000858,25.73,25.73,25.73,25.73,25.73,0,0.000000e+00,3,NaN,0.034128,25.042,25.730,0.000000
1410,2015-10-29,sz.000858,25.73,25.73,25.73,25.73,25.73,0,0.000000e+00,3,NaN,0.034128,25.060,25.730,0.000000
1411,2015-10-30,sz.000858,25.73,25.73,25.73,25.73,25.73,0,0.000000e+00,3,NaN,0.034128,25.098,25.730,0.000000
2738,2021-04-12,sz.000858,265.20,267.36,260.00,260.99,265.30,13323255,3.495255e+09,3,0.3510,0.747189,NaN,270.996,0.073384
2739,2021-04-13,sz.000858,261.26,268.22,261.26,262.90,260.99,13875729,3.670700e+09,3,0.3656,0.752978,NaN,266.624,0.076427
2740,2021-04-14,sz.000858,262.91,267.61,261.25,263.78,262.90,14817752,3.916715e+09,3,0.3904,0.755645,NaN,265.210,0.081615
2741,2021-04-15,sz.000858,261.31,262.99,249.32,255.80,263.78,28595938,7.267459e+09,3,0.7534,0.731458,NaN,261.754,0.157505
2742,2021-04-16,sz.000858,255.80,263.63,255.80,261.80,255.80,18754302,4.877111e+09,3,0.4941,0.749644,NaN,261.054,0.103298


In [20]:
stock['avg_price']

0           NaN
1           NaN
2           NaN
3           NaN
4        30.822
         ...   
2738    270.996
2739    266.624
2740    265.210
2741    261.754
2742    261.054
Name: avg_price, Length: 2743, dtype: float64

In [14]:
threshold_flat = 0.03
def flat_or_not(x):
    if x >= threshold_flat:
        return 2
    elif x <= -threshold_flat:
        return 1
    else:
        return 0

stock['label'] = ((stock['future_price'] - stock['avg_price']) / stock['avg_price']).apply(flat_or_not)

In [17]:
stock['label'][10:30]

10    0
11    1
12    1
13    1
14    1
15    1
16    1
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
Name: label, dtype: int64

In [116]:
stock['label'].value_counts()

2    1073
1     896
0     775
Name: label, dtype: int64

In [107]:
n = len(stock)
x = np.array([stock['close_nomalized'][i:i+history_period] for i in range(n-history_period-predict_period+1)]).reshape(-1, 20, 20) # 输入 400天
x = x[:, :, :, np.newaxis]
y = stock['label'][history_period-1:].values[:-predict_period]  

In [108]:
len(y), len(x)

(2340, 2340)

In [117]:
threshold_flat

0.01

In [118]:
x[]

array([[[0.33895069],
        [0.3438002 ],
        [0.3489225 ],
        [0.34364865],
        [0.35168066],
        [0.35046828],
        [0.34234535],
        [0.35737884],
        [0.36204649],
        [0.37138181],
        [0.38502107],
        [0.38620313],
        [0.38502107],
        [0.37535235],
        [0.37714061],
        [0.38050496],
        [0.37804989],
        [0.37213954],
        [0.34792229],
        [0.35486315]],

       [[0.35295366],
        [0.35483284],
        [0.36701725],
        [0.36507744],
        [0.37089686],
        [0.36486527],
        [0.36989664],
        [0.36998757],
        [0.36259206],
        [0.36168278],
        [0.349559  ],
        [0.3585306 ],
        [0.3556512 ],
        [0.35316582],
        [0.35395387],
        [0.3527718 ],
        [0.35831843],
        [0.35055921],
        [0.35107447],
        [0.34198163]],

       [[0.34446701],
        [0.34713424],
        [0.34204225],
        [0.33358591],
        [0.34470948],
      